# Projeto Ciencia de Dados: Carteira de Ações com Inteligência Artificial

- Projeto Inspiração: https://medium.com/swlh/teaching-a-machine-to-trade-stocks-like-warren-buffett-part-i-445849b208c6
- Resumo Fundamentos Empresas BR: https://fundamentus.com.br/index.php

### Criar Dicionários de Dataframes com cada empresa

In [1]:
import pandas as pd
import os
empresas = ["ABEV3", "AZUL4", "BTOW3", "B3SA3", "BBSE3", "BRML3", "BBDC4", "BRAP4", "BBAS3", "BRKM5", "BRFS3", "BPAC11", "CRFB3", "CCRO3", "CMIG4", "HGTX3", "CIEL3", "COGN3", "CPLE6", "CSAN3", "CPFE3", "CVCB3", "CYRE3", "ECOR3", "ELET6", "EMBR3", "ENBR3", "ENGI11", "ENEV3", "EGIE3", "EQTL3", "EZTC3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "NTCO3", "HAPV3", "HYPE3", "IGTA3", "GNDI3", "ITSA4", "ITUB4", "JBSS3", "JHSF3", "KLBN11", "RENT3", "LCAM3", "LAME4", "LREN3", "MGLU3", "MRFG3", "BEEF3", "MRVE3", "MULT3", "PCAR3", "PETR4", "BRDT3", "PRIO3", "QUAL3", "RADL3", "RAIL3", "SBSP3", "SANB11", "CSNA3", "SULA11", "SUZB3", "TAEE11", "VIVT3", "TIMS3", "TOTS3", "UGPA3", "USIM5", "VALE3", "VVAR3", "WEGE3", "YDUQ3"]

# Dicionario referência
""" fundamentos = {
    "ABEV3": balanco_dre_abev3,
    "MGLU3": balanco_dre_mglu3
} """
fundamentos = {}
arquivos = os.listdir("balancos")

for arquivo in arquivos:
    nome = arquivo[-9:-4]

    if "11" in nome:
        nome = arquivo[-10:-4]

    if nome in empresas:
        print(nome)
        # Pegar balanço da empresa
        balanco = pd.read_excel(f'balancos/{arquivo}', sheet_name=0)

        # Tornar a primeira coluna o titulo da empresa
        balanco.iloc[0,0] = nome

        # Tornar primeira linha cabeçalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]

        # Tornar a 1ª coluna em indice
        balanco = balanco.set_index(nome)

        dre = pd.read_excel(f'balancos/{arquivo}', sheet_name=1)

        # Tornar a primeira coluna o titulo da empresa
        dre.iloc[0,0] = nome

        # Tornar primeira linha cabeçalho
        dre.columns = dre.iloc[0]
        dre = dre[1:]

        # Tornar a 1ª coluna em indice
        dre = dre.set_index(nome)
        
        fundamentos[nome] = balanco.append(dre)
        



ABEV3
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local

AZUL4
WARNING *** file size (30309) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (30309) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
B3SA3
WARNING *** file size (64274) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64274) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
BBAS3
WARNING *** file size (68412) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (68412) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
BBDC4
WARNING *** file size (68284) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 b

C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local


WARNING *** file size (64136) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64136) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
CMIG4
WARNING *** file size (64388) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64388) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
COGN3
WARNING *** file size (64388) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64388) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
CPFE3
WARNING *** file size (64412) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SS

...

C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local


WARNING *** file size (64308) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64308) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
SANB11
WARNING *** file size (68210) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (68210) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
SBSP3
WARNING *** file size (63832) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (63832) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
SULA11
WARNING *** file size (64292) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local

TOTS3
WARNING *** file size (64280) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64280) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
UGPA3
WARNING *** file size (64320) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64320) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
USIM5
WARNING *** file size (64286) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64286) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
VALE3
WARNING *** file size (64278) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 b

C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)
C:\Users\breno\AppData\Local\Temp\ipykernel_15144\1154866316.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fundamentos[nome] = balanco.append(dre)


### Pegar Preços das Ações nas Datas Correspondentes

In [2]:
cotacoes_df = pd.read_excel("Cotacoes.xlsx")
cotacoes ={}
for empresa in cotacoes_df["Empresa"].unique():
    cotacoes[empresa] = cotacoes_df.loc[cotacoes_df['Empresa'] == empresa, :]


In [3]:
print(len(cotacoes))

77


### Remover empresas que tem cotações vazias da análise (mesmo após o tratamento que fizemos na hora de pegar as cotações)

In [4]:
for empresa in empresas:
    if cotacoes[empresa].isnull().values.any():
        cotacoes.pop(empresa)
        fundamentos.pop(empresa)
empresas = list(cotacoes.keys())
print(len(empresas))

65


### Juntar fundamentos com Preço da Ação

In [5]:
# No dataframe mover datas para o indice
# NO data frame fundamentos trocar linhas por colunas
    #Trocar datas por formato de data do python
    # Juntar os jundamentos com a coluna Adj Close das cotações
for empresa in fundamentos:
    tabela = fundamentos[empresa].T
    tabela.index = pd.to_datetime(tabela.index, format="%d/%m/%Y")
    tabela_cotacao = cotacoes[empresa].set_index("Date")
    tabela_cotacao = tabela_cotacao[["Adj Close"]]

    tabela = tabela.merge(tabela_cotacao, right_index=True, left_index=True)
    tabela.index.name = empresa
    fundamentos[empresa] = tabela
display(fundamentos["ABEV3"])


,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,Outros Ativos Circulantes,...,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período,Adj Close
ABEV3,,,,,,,,,,,,,,,,,,,,,
2020-12-31,125196574.72,35342614.528,17090335.744,1700028.032,4303137.792,7605904.896,0,3287141.888,293552,1062513.984,...,NaN,NaN,8464308.224,93644,-1667579.008,NaN,NaN,-104363.008,6786009.6,15.575767
2020-09-30,127056781.312,39098793.984,21660450.816,1442923.008,4156922.88,7341836.8,0,3472880.896,298303.008,725478.976,...,NaN,NaN,2431040,-415548,343476,NaN,NaN,-84167,2274800.896,12.159024
2020-06-30,121393004.544,36268126.208,17438951.424,44823,4201583.104,7771618.816,0,5390340.096,356411.008,1064401.984,...,NaN,NaN,1108173.056,-431944.992,595088,NaN,NaN,-44517,1226798.976,13.710416
2020-03-31,113687560.192,31086356.48,13204471.808,44823,3394298.112,7752655.872,0,5266407.936,447473.984,976225.024,...,NaN,NaN,1490919.936,-295032.992,15365,NaN,NaN,-119468,1091783.936,11.557861
2019-12-31,101742944.256,27621136.384,11900665.856,14558,4495524.864,5978556.928,0,4074127.104,512532,645171.968,...,NaN,NaN,3755047.68,1317924.992,-853995.008,NaN,NaN,-119273.008,4099704.832,18.102791
2019-09-30,102475317.248,29004025.856,15016913.92,14304,4236257.024,5978811.904,0,2545863.936,293060,918814.976,...,NaN,NaN,2826416.896,-649270.016,427238.016,NaN,NaN,-106708,2497677.056,18.187490
2019-06-30,98738978.816,27356035.072,14233403.392,14038,4103118.08,6197564.928,0,1542556.032,507216,758140.992,...,NaN,NaN,2980011.008,-807523.968,443414.016,NaN,NaN,-95168,2520732.928,16.902554
2019-03-31,96905535.488,26194137.088,12822524.928,13772,3980359.936,6032743.936,0,1980342.016,591273.984,773121.024,...,NaN,NaN,3381529.088,-979185.024,346724,NaN,NaN,-87218,2661850.112,15.901062
2018-12-31,94126137.344,25329604.608,11463497.728,13391,4879256.064,5401793.024,0,2148713.984,741222.016,681731.008,...,NaN,NaN,4591409.152,33055.072,-1160932.992,NaN,NaN,-103125.992,3360404.736,14.531096


### Tratar colunas
    
1. Vamos pegar apenas empresas que possuem as mesmas colunas
2. Ajeitar colunas com nome repetido
3. Analisar valores vazios nas colunas

#### 1. Remover da análise colunas que não existem em alguma tabela

In [6]:
colunas = list(fundamentos["ABEV3"].columns)

for empresa in empresas:
    if set(colunas) != set(fundamentos[empresa].columns):
        fundamentos.pop(empresa)



####  2. Ajeitando colunas com o mesmo nome

In [7]:
texto_colunas = ";".join(colunas)

colunas_modificadas = []
for coluna in colunas:
    if colunas.count(coluna) == 2 and coluna not in colunas_modificadas:
        texto_colunas = texto_colunas.replace(";" + coluna + ";",";" + coluna + "_1;",1)
        colunas_modificadas.append(coluna)
colunas = texto_colunas.split(";")


In [8]:
# Implementar as colunas nas tabelas
for empresa in fundamentos:
    fundamentos[empresa].columns = colunas

#### 3. Analisar valores vazios nas colunas

In [9]:
valores_vazios = dict.fromkeys(colunas, 0)
total_linhas = 0
for empresa in fundamentos:
    tabela = fundamentos[empresa]
    total_linhas += tabela.shape[0]
    for coluna in colunas:
        qtde_vazios = pd.isnull(tabela[coluna]).sum()
        valores_vazios[coluna] += qtde_vazios


In [10]:
remover_colunas = []
for coluna in valores_vazios:
    if valores_vazios[coluna] > 50:
        remover_colunas.append(coluna)

for empresas in fundamentos:
    fundamentos[empresas] = fundamentos[empresas].drop(remover_colunas, axis=1)
    fundamentos[empresas] = fundamentos[empresas].ffill()

In [11]:
fundamentos["ABEV3"].shape

(33, 72)

### Criando os rótulos: Comprar, Não Comprar ou Vender?

Não queremos saber quando vender, mas inclui essa categoria para conseguir identificar quando que o nosso modelo vai sugerir uma compra quando na verdade o melhor momento era vender. Isso significa que o modelo errou "mais" do que quando sugeriu comprar e simplesmente o certo era não comprar

Regra: 
1. Subiu mais do que o Ibovespa (ou caiu menos) -> Comprar (Valor = 2)
2. Subiu menos do que o Ibovespa até Ibovespa - 2% (ou caiu mais do que Ibovespa até Ibovespa -2%) -> Não Comprar (Valor = 1)
3. Subiu menos do que o Ibovespa - 2% (ou caiu mais do que Ibovespa -2%) -> Vender (Valor = 0)

In [ ]:
data_inicial = "12/20/2012"
data_final = "04/20/2021"

from pandas_datareader import data as web
df_ibov = web.DataReader('^BVSP', data_source='yahoo', start=data_inicial, end=data_final)

In [ ]:
import numpy as np
datas = fundamentos["ABEV3"].index
for data in datas:
    if data not in df_ibov.index:
        df_ibov.loc[data] = np.nan
df_ibov = df_ibov.sort_index()
df_ibov = df_ibov.ffill()
df_ibov = df_ibov.rename(columns={"Adj Close": "IBOV"})
for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].merge(df_ibov[["IBOV"]], left_index = True, right_index = True)


In [20]:
# Transformar indicadores em percentuais 
for empresa in fundamentos:
    fundamento = fundamentos[empresa]
    fundamento = fundamento.sort_index()
    for coluna in fundamento:
        if "Adj Close" in coluna or "IBOV" in coluna:
            fundamento["Adj Close"] = fundamento["Adj Close"].shift(-1) / fundamento["Adj Close"] - 1
            fundamento["IBOV"] = fundamento["IBOV"].shift(-1) / fundamento["IBOV"] - 1
            fundamento["Resultado"] = fundamento["Adj Close"] - fundamento["IBOV"]
            condicoes = [
                (fundamento["Resultado"] > 0),
                (fundamento["Resultado"] < 0) & (fundamento["Resultado"] >= -0.02),
                (fundamento["Resultado"] < -0.02) 
            ]
            valores = [2,1,0]
            fundamento["Decisao"] = np.select(condicoes, valores)
        else:
            condicoes = [
                (fundamento[coluna].shift(1) > 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] > 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] > 0),
                (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] == 0),
                
            ]
            valores = [
                -1,
                1,
                (fundamento[coluna].shift(1) - abs(fundamento[coluna])) / abs(fundamento[coluna].shift(1)),
                1,
                -1,
                1,
            ]
            fundamento[coluna] = np.select(condicoes, valores, default = fundamento[coluna] / fundamento[coluna].shift(1) - 1)
fundamentos[empresa] = fundamento
display(fundamentos["ABEV3"])

,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber_1,Estoques_1,Ativos Biológicos_1,Tributos a Recuperar,Despesas Antecipadas_1,Outros Ativos Circulantes,...,Resultado da Equivalência Patrimonial,Financeiras,Receitas Financeiras,Despesas Financeiras,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Lucro/Prejuízo do Período,Adj Close,IBOV
2020-12-31,1.251966e+08,3.534261e+07,1.709034e+07,1700028.032,4303137.792,7605904.896,0,3287141.888,293552.000,1062513.984,...,-9428,1040928.960,2144160.000,-1103231.104,8464308.224,93644.000,-1667579.008,6786009.600,15.575767,120062.0
2020-09-30,1.270568e+08,3.909879e+07,2.166045e+07,1442923.008,4156922.880,7341836.800,0,3472880.896,298303.008,725478.976,...,-11002,-1144775.936,369476.992,-1514253.056,2431040.000,-415548.000,343476.000,2274800.896,12.159024,94603.0
2020-06-30,1.213930e+08,3.626813e+07,1.743895e+07,44823.000,4201583.104,7771618.816,0,5390340.096,356411.008,1064401.984,...,-16237,-793734.976,236606.000,-1030340.992,1108173.056,-431944.992,595088.000,1226798.976,13.710416,95056.0
2020-03-31,1.136876e+08,3.108636e+07,1.320447e+07,44823.000,3394298.112,7752655.872,0,5266407.936,447473.984,976225.024,...,-6617,-1536866.944,245776.000,-1782642.944,1490919.936,-295032.992,15365.000,1091783.936,11.557861,73020.0
2019-12-31,1.017429e+08,2.762114e+07,1.190067e+07,14558.000,4495524.864,5978556.928,0,4074127.104,512532.000,645171.968,...,-11195,-1564263.936,234065.056,-1798328.832,3755047.680,1317924.992,-853995.008,4099704.832,18.102791,120062.0
2019-09-30,1.024753e+08,2.900403e+07,1.501691e+07,14304.000,4236257.024,5978811.904,0,2545863.936,293060.000,918814.976,...,-8248,-305798.016,867204.992,-1173003.008,2826416.896,-649270.016,427238.016,2497677.056,18.187490,104745.0
2019-06-30,9.873898e+07,2.735604e+07,1.423340e+07,14038.000,4103118.080,6197564.928,0,1542556.032,507216.000,758140.992,...,-731,-567436.992,249903.008,-817340.032,2980011.008,-807523.968,443414.016,2520732.928,16.902554,120062.0
2019-03-31,9.690554e+07,2.619414e+07,1.282252e+07,13772.000,3980359.936,6032743.936,0,1980342.016,591273.984,773121.024,...,-2136,-672067.968,287692.992,-959761.024,3381529.088,-979185.024,346724.000,2661850.112,15.901062,120062.0
2018-12-31,9.412614e+07,2.532960e+07,1.146350e+07,13391.000,4879256.064,5401793.024,0,2148713.984,741222.016,681731.008,...,1140,-1618994.944,19940.992,-1638935.808,4591409.152,33055.072,-1160932.992,3360404.736,14.531096,120062.0
2018-09-30,9.569104e+07,2.620002e+07,1.222767e+07,13008.000,4285633.024,5349087.232,0,2215026.944,439839.008,1669756.032,...,-3272,-611080.000,245288.992,-856369.024,2754401.024,-475184.992,612889.984,2831159.040,16.978544,120062.0


### Hora de tornar tudo 1 dataframe só

### Análise Exploratória

#### 1. Quantidade de Respostas em cada Tipo de Decisão

#### 2. Correlação

#### Vamos remover Todas as Colunas "já explicadas" pelo Ativo Total

### Vamos partir para Feature Selection

Será que todas essas features são importantes mesmo para o nosso modelo? Muitas features nem sempre é bom, se pudermos reduzir sem perder eficiência do nosso modelo, melhor

Aqui temos 2 alternativas:

1. Seguir com todas as features e depois tentar melhorar o nosso modelo
2. Usar algum critério para selecionar as melhores features para prever e criar o modelo a partir apenas dessa seleção menor de features

Vou seguir com a opção 2, porque é mais rápida e, caso dê certo, facilita a nossa vida. Se der errado, a gente volta aqui e refaz o processo

### Aplicação do StandardScaler para melhorar nossos modelos de MachineLearning

### Separação dos dados em treino e teste

### Criação de um Dummy Classifier (Uma baseline para ver se os nossos modelos são melhores do que puro chute)

### Métricas de Avaliação

- Precisão vai ser nossa métrica principal
- Recall pode ser útil, mas precisão no caso de ações é mt mais importante.

Explicação: Foto dos Gatos e Cachorros na Wikipedia: https://en.wikipedia.org/wiki/Precision_and_recall

### Modelos que vamos testar
- AdaBoost
- Decision Tree
- Random Forest
- ExtraTree
- Gradient Boost
- K Nearest Neighbors (KNN)
- Logistic Regression
- Naive Bayes
- Support Vector Machine (SVM)
- Rede Neural

### Agora vamos ao tunning do modelo

- é bom sempre incluir no tuning os parâmetros "padrões" do modelo, se não poder ser que vc só encontre resultados piores

### Vamos seguir então com o Modelo BLA tunado para ser o nosso modelo

- Agora ao que interessa...será que vamos conseguir escolher boas ações e ter uma carteira rentável?
- Para isso, temos que pegar o último Trimestre (que não usamos para treinar o modelo nem testar) e ver como ele teria saído até o momento em que gravo esse vídeo...

### Repetindo os passos com a base fundamentos só que pegando apenas o último tri de cada empresa

### Agora fazemos a previsão de decisões